In [36]:
#
# Import all packages
#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pydot

## Models used in the assignment
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, validation_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc, roc_auc_score
from io import StringIO
from sklearn.tree import export_graphviz

import warnings
# To ignore any future warnings
warnings.filterwarnings("ignore")

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    

# read the dataset and set skipinitialspace to true to be able to .replace
df = pd.read_csv('./MENS_CLOTHING_SALES_2018.csv', skipinitialspace=True)
print(df.info())

print(df['TotalInvestment'].isna().sum())
print(df['Sales'].isna().sum())
print(df['AnnualSales'].isna().sum())


## 3 attributes with 15 attributes NaN, therefore removed
df = df.dropna(subset=["TotalInvestment", "Sales", "AnnualSales"], how='all')
print(df.info())


rs = 10

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
StoreCode          400 non-null int64
AnnualSales        385 non-null float64
Sales              385 non-null float64
SFloorSize         400 non-null int64
TotalInvestment    385 non-null float64
dtypes: float64(3), int64(2)
memory usage: 15.7 KB
None
15
15
15
<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 399
Data columns (total 5 columns):
StoreCode          385 non-null int64
AnnualSales        385 non-null float64
Sales              385 non-null float64
SFloorSize         385 non-null int64
TotalInvestment    385 non-null float64
dtypes: float64(3), int64(2)
memory usage: 18.0 KB
None


In [58]:
#        #
# Task 1 #
#        #
print(color.BOLD + "Task 1 Question 1" + color.END)
print(color.BOLD + "Can you identify data quality issues in this dataset such as unusual data types, missing values and others?" + color.END)
print("\nThere are 15 rows that have 3 attributes missing, which is 60% missing values, \
therefore not reliable to impute and is removed from df")


print(color.BOLD + "\n\nTask 1 Question 2" + color.END)
print(color.BOLD +"What variables did you include in the analysis and what were their roles and measurement level set? Justify your choice."+ color.END)
print("\nSince all the variables have reasonable values with no missing records, 4 attributes are used.")
print("We can use either AnnualSales or Sales to determine which provides better cluster")

print(color.BOLD + "\n\nTask 1 Question 3" + color.END)
print(color.BOLD + "Identify a store that is underperforming in sales. Based on your reporting, the company does not \
want to focus their efforts on this store. Now onwards, the selected store should not be part of analysis" + color.END)


print(df.info())

Task 1 Question 1
Can you identify data quality issues in this dataset such as unusual data types, missing values and others?

There are 15 rows that have 3 attributes missing, which is 60% missing values, therefore not reliable to impute and is removed from df


Task 1 Question 2
What variables did you include in the analysis and what were their roles and measurement level set? Justify your choice.

Since all the variables have reasonable values with no missing records, 4 attributes are used.
We can use either AnnualSales or Sales to determine which provides better cluster


Task 1 Question 3
Identify a store that is underperforming in sales. Based on your reporting, the company does not want to focus their efforts on this store. Now onwards, the selected store should not be part of analysis
<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 399
Data columns (total 5 columns):
StoreCode          385 non-null int64
AnnualSales        385 non-null float64
Sales         

In [57]:
#        #
# Task 2 #
#        #

print(color.BOLD + "Task 2 Question 1" + color.END)
print(color.BOLD + "Build a default clustering model with K= 3 and answer the followings:\
a. How many records are assigned into each cluster?\
b. Plot the cluster distribution using pairplot. Explain key characteristics of each cluster/\
segment." + color.END)

print(color.BOLD + "\nTask 2 Question 2" + color.END)
print(color.BOLD + "What is the effect of using the standardization method on the model above? Does the variable\
normalization process enable a better clustering solution?" + color.END)

print(color.BOLD + "\nTask 2 Question 3" + color.END)
print(color.BOLD + "Interpret the (best out of 2.1 and 2.2) cluster analysis outcome. In other words, characterize the\
nature of each cluster by giving it a descriptive label by using distplot" + color.END)


Task 2 Question 1
Build a default clustering model with K= 3 and answer the followings:a. How many records are assigned into each cluster?b. Plot the cluster distribution using pairplot. Explain key characteristics of each cluster/segment.

Task 2 Question 2
What is the effect of using the standardization method on the model above? Does the variablenormalization process enable a better clustering solution?

Task 2 Question 3
Interpret the (best out of 2.1 and 2.2) cluster analysis outcome. In other words, characterize thenature of each cluster by giving it a descriptive label by using distplot


In [59]:
#        #
# Task 3 #
#        #

print(color.BOLD + "Task 3 Question 1" + color.END)
print(color.BOLD + "" + color.END)

print(color.BOLD + "\nTask 3 Question 2" + color.END)
print(color.BOLD + "" + color.END)

print(color.BOLD + "\nTask 3 Question 3" + color.END)
print(color.BOLD + "" + color.END)


Task 3 Question 1


Task 3 Question 2


Task 3 Question 3



In [ ]:
#        #
# Task 4 #
#        #

print(color.BOLD + "Task 4 Question 1" + color.END)
print(color.BOLD + "" + color.END)

print(color.BOLD + "\nTask 4 Question 2" + color.END)
print(color.BOLD + "" + color.END)

print(color.BOLD + "\nTask 4 Question 3" + color.END)
print(color.BOLD + "" + color.END)

In [ ]:
#        #
# Task 5 #
#        #

print(color.BOLD + "Task 5 Question 1" + color.END)
print(color.BOLD + "" + color.END)

print(color.BOLD + "\nTask 5 Question 2" + color.END)
print(color.BOLD + "" + color.END)

print(color.BOLD + "\nTask 5 Question 3" + color.END)
print(color.BOLD + "" + color.END)